## 数据集划分

In [33]:
import json
from sklearn.model_selection import train_test_split

# 加载原始数据
with open('data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 划分索引保持原始格式
train_idx, test_idx = train_test_split(
    range(len(data)),
    test_size=0.2,
    random_state=42,
    stratify=[1 if 'hate' in d['output'] else 0 for d in data]
)

# 按索引划分数据
train_data = [data[i] for i in train_idx]
test_data = [data[i] for i in test_idx]

# 统计数据量
print(f"训练集数据量: {len(train_data)}条")
print(f"测试集数据量: {len(test_data)}条")

# 保存划分后的数据（保持原始JSON格式）
with open('train.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

with open('test.json', 'w', encoding='utf-8') as f:
    json.dump(test_data, f, ensure_ascii=False, indent=2)

训练集数据量: 3200条
测试集数据量: 800条


## 数据统计

In [4]:
import json
import pandas as pd
from collections import Counter

def parse_quadruples(output_str):
    """解析output字段为四元组列表"""
    quads = []
    for quad in output_str.split('[SEP]'):
        parts = [p.strip() for p in quad.split('|')]
        if len(parts) >= 4:
            quads.append({
                'target': parts[0],
                'argument': parts[1],
                'target_group': parts[2],
                'hateful': parts[3].replace('[END]', '').strip()
            })
    return quads

def analyze_data(data):
    stats = {
        'total_samples': len(data),
        'hate_quads': 0,
        'non_hate_quads': 0,
        'avg_quads_per_sample': 0
    }
    
    target_groups = []
    hate_status = []
    
    for sample in data:
        quads = parse_quadruples(sample['output'])
        stats['avg_quads_per_sample'] += len(quads)
        for quad in quads:
            target_groups.append(quad['target_group'])
            if quad['hateful'] == 'hate':
                stats['hate_quads'] += 1
            else:
                stats['non_hate_quads'] += 1
    
    stats['avg_quads_per_sample'] /= stats['total_samples']
    stats['target_group_dist'] = dict(Counter(target_groups))
    
    return stats

if __name__ == '__main__':
    with open('train.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    stats = analyze_data(data)
    
    # 保存基础统计
    pd.DataFrame.from_dict(stats, orient='index').to_excel('output/basic_stats.xlsx')
    
    # 保存目标群体分布
    pd.DataFrame.from_dict(
        stats['target_group_dist'], 
        orient='index', 
        columns=['count']
    ).to_excel('output/target_group_dist.xlsx')

## 词云图

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def load_stopwords(filepath):
    """加载屏蔽词文件"""
    with open(filepath, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f]

def extract_argument(output_str):
    """提取两个|之间的Argument内容"""
    parts = output_str.split(' | ')
    if len(parts) >= 2:
        return parts[1]  # 返回第二个部分（Argument）
    return ""

def generate_wordcloud(texts, save_path, stopwords=None):
    """生成词云，支持屏蔽词过滤"""
    wordcloud = WordCloud(
        font_path='simhei.ttf',
        width=800,
        height=600,
        background_color='white',
        stopwords=stopwords,
        max_font_size=120
    ).generate(' '.join(texts))
    
    plt.figure(figsize=(10,8))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.savefig(save_path)
    plt.close()

def analyze_hate_speech():
    # 1. 加载数据
    with open('train.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 2. 提取仇恨言论的Argument部分
    hate_arguments = [extract_argument(d['output']) for d in data if 'hate' in d['output']]
    
    # 3. 加载屏蔽词
    stopwords = load_stopwords('stopwords.txt')
    
    # 4. 生成词云
    generate_wordcloud(hate_arguments, 'output/wordcloud.png', stopwords)
    print("仇恨言论词云已生成！")

if __name__ == '__main__':
    analyze_hate_speech()

仇恨言论词云已生成！


# 提取类别目标词

类别和类别之间可能包含的词汇会有相同的词汇，这种情况的话，在全部写入后，帮我遍历一遍，然后去除重复的，去除规则是这样的：

- 1.当两个类别中的分类数不一样，保留分类数多的，如“Region”和“Region,others”保留后者中的那一条内容

- 2.当两个类别中的分类数一样时，保留长度小的那个，如“Region”和“Sexism”含有的元素数量分别是20,13，那么保留后者中的那一条内容

In [ ]:
import json
from collections import defaultdict

# 需要排除的词语列表
exclude_words = [
    '人','群体','我们','你们','这些人','某些人','部分人','广告','弹幕',
    '哥们','西方紫苯','群体','组织','机构','政府','社会','国家','民族', 
    '种族','这个','那个','这','那','他们','他','她','她们','它们',
    '它','那些人','有些人','有人','这个吧','那个吧','这吧','那吧','他们吧',
    '他吧','她吧','她们吧','它们吧','它吧','这些人吧','某些人吧','部分人吧',
    ,'相当一部分人'
]

# 加载训练数据
with open('./train.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 初始化字典
TARGET_KEYWORDS = defaultdict(list)

# 遍历数据统计关键词部分
for item in data:
    for quad in item['output'].split(' [SEP] '):
        parts = quad.split(' | ')
        if len(parts) >= 4:
            target = parts[0].strip()
            group = parts[2].strip()
            if target and group and target not in exclude_words:
                TARGET_KEYWORDS[group].append(target)

# 去重并输出到文件
def remove_duplicates(keywords_dict):
    # 创建一个反向映射：词汇 -> 类别列表
    word_to_categories = defaultdict(list)
    for category, words in keywords_dict.items():
        for word in words:
            word_to_categories[word].append(category)
    
    # 处理重复词汇
    for word, categories in word_to_categories.items():
        if len(categories) > 1:
            # 按规则排序：1. 类别数量多的优先 2. 类别数量相同时，保留元素数量少的
            categories.sort(
                key=lambda c: (-len(c.split(',')), len(keywords_dict[c]))
            )
            # 保留第一个类别，从其他类别中删除该词汇
            for category in categories[1:]:
                if word in keywords_dict[category]:
                    keywords_dict[category].remove(word)
    
    return keywords_dict

# 去重处理
TARGET_KEYWORDS = remove_duplicates(TARGET_KEYWORDS)

# 输出到文件
with open('h:\\project\\Hate_Identification\\data\\target_keywords.txt', 'w', encoding='utf-8') as f:
    f.write('TARGET_KEYWORDS = {\n')
    for i, (k, v) in enumerate(TARGET_KEYWORDS.items()):
        f.write(f'    "{k}": {v}')
        if i < len(TARGET_KEYWORDS) - 1:
            f.write(',\n\n')
        else:
            f.write('\n\n')
    f.write('}\n')
print("关键词已保存到 target_keywords.txt 文件中")

关键词已保存到 target_keywords.txt 文件中。
